## Preprocessing

In [40]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [41]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [42]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [43]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [44]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [45]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C6100        1
C1820        1
C1900        1
C2150        1
Name: count, Length: 71, dtype: int64


In [46]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts[classification_counts > 1])

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1256        2
C1246        2
C1234        2
C1267        2
Name: count, dtype: int64


In [47]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [48]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [49]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL']).values
y = application_df['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [51]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

c:\Users\fitse\anaconda3\envs\fitenv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [53]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=2)

Epoch 1/100
858/858 - 2s - 2ms/step - accuracy: 0.7204 - loss: 0.5699
Epoch 2/100
858/858 - 2s - 2ms/step - accuracy: 0.7301 - loss: 0.5530
Epoch 3/100
858/858 - 1s - 1ms/step - accuracy: 0.7308 - loss: 0.5516
Epoch 4/100
858/858 - 1s - 1ms/step - accuracy: 0.7322 - loss: 0.5495
Epoch 5/100
858/858 - 1s - 1ms/step - accuracy: 0.7315 - loss: 0.5477
Epoch 6/100
858/858 - 1s - 2ms/step - accuracy: 0.7323 - loss: 0.5471
Epoch 7/100
858/858 - 2s - 2ms/step - accuracy: 0.7338 - loss: 0.5463
Epoch 8/100
858/858 - 1s - 1ms/step - accuracy: 0.7338 - loss: 0.5459
Epoch 9/100
858/858 - 1s - 2ms/step - accuracy: 0.7332 - loss: 0.5450
Epoch 10/100
858/858 - 1s - 1ms/step - accuracy: 0.7349 - loss: 0.5445
Epoch 11/100
858/858 - 1s - 1ms/step - accuracy: 0.7335 - loss: 0.5444
Epoch 12/100
858/858 - 1s - 2ms/step - accuracy: 0.7341 - loss: 0.5442
Epoch 13/100
858/858 - 1s - 1ms/step - accuracy: 0.7350 - loss: 0.5434
Epoch 14/100
858/858 - 1s - 1ms/step - accuracy: 0.7349 - loss: 0.5426
Epoch 15/100
85

In [54]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7252 - loss: 0.5669
Loss: 0.5669056177139282, Accuracy: 0.725218653678894


In [55]:
# Export our model to HDF5 file
nn.save("charity_model.h5")

## **Step 3 Optimize the MOdel. Compile, Trian and Evaluate the Model More**

In [56]:
# Define the optimized model
nn = tf.keras.models.Sequential()

In [57]:
# First hidden layer with increased neurons and LeakyReLU activation
nn.add(tf.keras.layers.Dense(units=128, input_dim=X_train.shape[1]))
nn.add(tf.keras.layers.LeakyReLU(alpha=0.01))

# Second hidden layer with increased neurons and tanh activation
nn.add(tf.keras.layers.Dense(units=64, activation='tanh'))

# Third hidden layer with increased neurons and LeakyReLU activation
nn.add(tf.keras.layers.Dense(units=32))
nn.add(tf.keras.layers.LeakyReLU(alpha=0.01))

# Fourth hidden layer (newly added) with tanh activation
nn.add(tf.keras.layers.Dense(units=16, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

c:\Users\fitse\anaconda3\envs\fitenv\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 128)            │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,513 (64.50 KB)

 Trainable params: 16,513 (64.50 KB)

 Non-trainable params: 0 (0.00 B)

In [66]:
# Compile the model 
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [67]:
# Create a callback that saves the model's weights every five epochs
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("optimized_weights.weights.h5", save_weights_only=True, save_freq=5*len(X_train)//32)

In [68]:
nn.fit(X_train_scaled, y_train, epochs=120, batch_size=32, verbose=2, callbacks=[checkpoint_cb])

Epoch 1/120
858/858 - 4s - 4ms/step - accuracy: 0.7380 - loss: 0.5425
Epoch 2/120
858/858 - 2s - 3ms/step - accuracy: 0.7384 - loss: 0.5408
Epoch 3/120
858/858 - 2s - 3ms/step - accuracy: 0.7386 - loss: 0.5407
Epoch 4/120
858/858 - 2s - 2ms/step - accuracy: 0.7381 - loss: 0.5411
Epoch 5/120
858/858 - 2s - 2ms/step - accuracy: 0.7385 - loss: 0.5406
Epoch 6/120
858/858 - 2s - 3ms/step - accuracy: 0.7384 - loss: 0.5399
Epoch 7/120
858/858 - 2s - 2ms/step - accuracy: 0.7387 - loss: 0.5401
Epoch 8/120
858/858 - 2s - 2ms/step - accuracy: 0.7394 - loss: 0.5397
Epoch 9/120
858/858 - 2s - 2ms/step - accuracy: 0.7382 - loss: 0.5393
Epoch 10/120
858/858 - 2s - 2ms/step - accuracy: 0.7390 - loss: 0.5390
Epoch 11/120
858/858 - 2s - 2ms/step - accuracy: 0.7383 - loss: 0.5401
Epoch 12/120
858/858 - 2s - 2ms/step - accuracy: 0.7396 - loss: 0.5396
Epoch 13/120
858/858 - 2s - 2ms/step - accuracy: 0.7393 - loss: 0.5387
Epoch 14/120
858/858 - 2s - 2ms/step - accuracy: 0.7382 - loss: 0.5390
Epoch 15/120
85

In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7271 - loss: 0.5638
Loss: 0.563836395740509, Accuracy: 0.7271137237548828


In [70]:
nn.save("AlphabetSoupCharity_Optimization.h5")